# Example code to train VAE

In [29]:
from simulator import VariationalAutoencoder, save_dict_pickle, load_dict_pickle, SimulationDataScaler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr

### Load Simulated Data

In [2]:
simulated_data = load_dict_pickle('20200120_simulated_data.pickle')

In [4]:
list(simulated_data.keys())

['X_train_complete',
 'X_val_complete',
 'X_test_complete',
 'Y_train_complete',
 'Y_val_complete',
 'Y_test_complete',
 'X_train_um',
 'X_val_um',
 'X_test_um',
 'Y_train_um',
 'Y_val_um',
 'Y_test_um']

In [6]:
X_train_um = simulated_data['X_train_um']
Y_train_um = simulated_data['Y_train_um']
X_val_um = simulated_data['X_val_um']
Y_val_um = simulated_data['Y_val_um']

In [14]:
X_train_um.describe()

,resp,shock_indexes,systolics,temperature,urine,MAP,o2_24h_max,spo2_24hr_min,pulse,AGE_AT_START_OF_ENCOUNTER,...,height,weight,platelets,shock_index_age,anion_gap,pulse_pressure,window_num,albumin,protein_level,diastolics
count,49960.000000,49961.000000,49987.000000,48611.000000,37130.000000,49988.000000,49960.000000,49948.000000,49995.000000,50000.000000,...,32580.000000,42449.000000,43742.000000,49953.000000,43573.000000,49979.000000,50000.000000,29704.000000,29355.000000,49986.000000
mean,18.431267,0.714924,122.480316,36.788361,274.272186,86.151390,2.142308,92.862534,85.224884,57.526039,...,1.699483,83.152763,222.761078,40.215744,12.053883,0.443770,5.036349,3.703878,6.479697,67.993462
std,2.307730,0.183246,22.053869,0.265728,145.952515,14.096407,2.054200,2.749843,15.372195,14.193875,...,0.065402,14.961164,64.168648,14.070885,1.780867,0.050794,4.729579,0.474927,0.612516,11.067472
min,14.364316,0.214100,73.620064,36.071255,-55.800388,54.802345,-1.355333,61.645058,43.585056,8.474298,...,1.460719,34.275742,-80.985092,0.801388,6.791385,0.238849,-2.576122,1.702451,3.983111,39.285114
25%,17.073940,0.584319,104.082581,36.641098,175.959747,74.469139,0.751234,91.832855,73.670395,47.978536,...,1.652951,72.843140,183.248138,30.274240,10.804937,0.410636,2.847863,3.400325,6.089336,59.218407
50%,17.804399,0.695298,118.635956,36.734791,238.529449,84.281525,1.439410,93.233742,83.905266,59.766405,...,1.699363,81.219131,217.120224,38.329742,11.866473,0.443224,3.807496,3.738561,6.518030,66.474869
75%,18.948053,0.825283,137.710800,36.852093,333.332825,96.065742,2.853918,94.521477,95.448303,68.239954,...,1.746406,91.574539,255.831543,48.442451,13.086332,0.477468,5.330729,4.041468,6.915159,75.622864
max,45.023113,1.650467,211.602493,39.900089,1294.610840,146.211853,16.435431,99.100471,152.501236,93.911957,...,1.921419,174.343643,634.443787,110.932770,22.771687,0.640637,70.544189,5.074804,8.302896,121.379723


### Check Missingness

In [9]:
X_train_um.isna().mean()

resp                         0.00080
shock_indexes                0.00078
systolics                    0.00026
temperature                  0.02778
urine                        0.25740
MAP                          0.00024
o2_24h_max                   0.00080
spo2_24hr_min                0.00104
pulse                        0.00010
AGE_AT_START_OF_ENCOUNTER    0.00000
bun                          0.13058
calcium                      0.12912
chloride                     0.12788
co2                          0.12976
creatinine                   0.12732
glucose                      0.09140
hematocrit                   0.11732
hemoglobin                   0.12062
mean_corps_hgb               0.12254
mean_corps_hgb_conc          0.12288
mean_corps_hgb_vol           0.12364
mean_platelet_vol            0.13586
potassium                    0.11558
red_blood_cell_count         0.12384
red_cell_dist_width          0.12604
sodium                       0.11644
gcs                          0.22154
w

### Scale and Impute with mean
* Better imputation methods could be used here

In [12]:
data_scaler = SimulationDataScaler()

In [13]:
data_scaler.fit(X_train_um)

In [17]:
X_train_um_scaled, X_train_um_ind = data_scaler.transform(X_train_um)
X_val_um_scaled, X_val_um_ind = data_scaler.transform(X_val_um)

In [18]:
X_train_um_scaled.describe()

,resp,shock_indexes,systolics,temperature,urine,MAP,o2_24h_max,spo2_24hr_min,pulse,AGE_AT_START_OF_ENCOUNTER,...,height,weight,platelets,shock_index_age,anion_gap,pulse_pressure,window_num,albumin,protein_level,diastolics
count,5.000000e+04,5.000000e+04,5.000000e+04,5.000000e+04,5.000000e+04,5.000000e+04,50000.000000,5.000000e+04,5.000000e+04,5.000000e+04,...,5.000000e+04,5.000000e+04,5.000000e+04,5.000000e+04,5.000000e+04,5.000000e+04,5.000000e+04,5.000000e+04,5.000000e+04,5.000000e+04
mean,7.324219e-09,2.441406e-09,1.052856e-08,7.324219e-09,1.220703e-09,1.007080e-08,0.000000,2.441406e-09,1.586914e-08,-1.220703e-08,...,-1.098633e-08,7.629395e-09,1.098633e-08,7.324219e-09,-9.765625e-09,1.039505e-08,-3.662109e-09,-4.882812e-09,1.342773e-08,-8.544922e-09
std,9.996099e-01,9.996199e-01,9.998800e-01,9.860221e-01,8.617510e-01,9.998900e-01,0.999610,9.994898e-01,9.999600e-01,1.000010e+00,...,8.072255e-01,9.214103e-01,9.353381e-01,9.995399e-01,9.335296e-01,9.998000e-01,1.000010e+00,7.707736e-01,7.662322e-01,9.998700e-01
min,-1.762334e+00,-2.733105e+00,-2.215518e+00,-2.698680e+00,-2.261537e+00,-2.223925e+00,-1.702695,-1.135257e+01,-2.708803e+00,-3.455873e+00,...,-3.650757e+00,-3.266965e+00,-4.733614e+00,-2.801157e+00,-2.955055e+00,-4.034411e+00,-1.609561e+00,-4.214249e+00,-4.076025e+00,-2.593966e+00
25%,-5.879122e-01,-7.120336e-01,-8.339903e-01,-5.443461e-01,-5.286755e-01,-8.286207e-01,-0.676989,-3.735618e-01,-7.516282e-01,-6.726560e-01,...,-3.148487e-01,-5.847651e-01,-5.317143e-01,-7.057414e-01,-6.054935e-01,-6.520727e-01,-4.627278e-01,-1.247874e-01,-1.297121e-01,-7.927659e-01
50%,-2.711526e-01,-1.060090e-01,-1.733065e-01,-1.815041e-01,0.000000e+00,-1.321312e-01,-0.341537,1.339325e-01,-8.573830e-02,1.578422e-01,...,0.000000e+00,0.000000e+00,0.000000e+00,-1.327677e-01,0.000000e+00,-1.032614e-02,-2.598256e-01,0.000000e+00,0.000000e+00,-1.367543e-01
75%,2.236488e-01,6.012142e-01,6.903922e-01,2.214300e-01,1.249265e-01,7.031777e-01,0.345502,6.026543e-01,6.649721e-01,7.548344e-01,...,3.231322e-01,4.098270e-01,4.051115e-01,5.836623e-01,4.573106e-01,6.631775e-01,6.224296e-02,2.687464e-01,2.535732e-01,6.891508e-01
max,1.152306e+01,5.105456e+00,4.041153e+00,1.171029e+01,6.990989e+00,4.260736e+00,6.958070,2.268493e+00,4.376540e+00,2.563520e+00,...,3.393467e+00,6.095245e+00,6.415709e+00,5.025820e+00,6.018375e+00,3.875848e+00,1.385081e+01,2.886654e+00,2.976625e+00,4.823757e+00


### Fit Model

In [20]:
input_features_with_y = X_train_um_scaled.shape[1] + 1 # add 1 for target

In [21]:
vae = VariationalAutoencoder(input_features=input_features_with_y, codings_size=16, d1_size=256, d2_size=128, d3_size=64, d4_size=32, patience=7, optimizer='adam')

In [23]:
history = vae.fit(X_train_um_scaled,Y_train_um,X_val_um_scaled,Y_val_um
                  ,batch_size=512, epochs=100)

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 2s 48us/step - loss: 0.8927 - numeric_loss: 0.7158 - categorical_loss: 0.1789 - val_loss: 0.6902 - val_numeric_loss: 0.5318 - val_categorical_loss: 0.0749
Epoch 2/100
50000/50000 [==============================] - 2s 33us/step - loss: 0.6858 - numeric_loss: 0.5042 - categorical_loss: 0.0548 - val_loss: 0.6226 - val_numeric_loss: 0.4482 - val_categorical_loss: 0.0587
Epoch 3/100
50000/50000 [==============================] - 2s 33us/step - loss: 0.6391 - numeric_loss: 0.4485 - categorical_loss: 0.0522 - val_loss: 0.5859 - val_numeric_loss: 0.4071 - val_categorical_loss: 0.0553
Epoch 4/100
50000/50000 [==============================] - 2s 33us/step - loss: 0.6092 - numeric_loss: 0.4153 - categorical_loss: 0.0503 - val_loss: 0.5605 - val_numeric_loss: 0.3762 - val_categorical_loss: 0.0474
Epoch 5/100
50000/50000 [==============================] - 2s 33us/step - loss: 0.5904 - numer

### Evaluate Performance

In [25]:
### Add Y to X matrix for evaluation

X_val_um_scaled_with_y = X_val_um_scaled.copy()
X_val_um_scaled_with_y['Target'] = Y_val_um

X_val_um_with_y = X_val_um.copy()
X_val_um_with_y['Target'] = Y_val_um

In [24]:
_, _, codings_val = vae.predict_encoder(X_val_um_scaled,Y_val_um)
X_val_with_y_pred = vae.predict_decoder(codings_val)

In [26]:
corr_mat_pred = pd.DataFrame(X_val_with_y_pred).corr()
corr_mat_imp = pd.DataFrame(X_val_um_scaled_with_y).corr()
corr_mat_raw = pd.DataFrame(X_val_um_with_y).corr()

In [27]:
(np.sign(np.triu(corr_mat_pred)) == np.sign(np.triu(corr_mat_raw))).mean()


0.967391304347826

* The signs of the upper triangle of the correlation matricies match very well!

In [30]:
X_val_um_with_y_df = pd.DataFrame(X_val_um_with_y)
X_val_with_y_pred_df = pd.DataFrame(X_val_with_y_pred)

loss1_cors = []
for i in range(0,X_val_um_with_y_df.shape[1]):
    use = X_val_um_with_y_df.iloc[:,i].notna().values
    r,p = pearsonr( X_val_um_with_y_df.loc[use].iloc[:,i].values,
                   X_val_with_y_pred_df.loc[use].iloc[:,i].values )
    loss1_cors.append(r)


In [31]:
np.mean(loss1_cors)

0.8751452669550324

* The average Pearson correlation between the reconstructions and the raw variables is strong

### Save model

In [33]:
out_model = 'example_model.h5'
vae.save(out_model)

In [34]:
!ls -lh $out_model

-rw-r--r-- 1 cgillies 4294967294 476K Jan 20 16:51 example_model.h5
